In [275]:
import time
import scipy as sp
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# data = pd.read_csv('features.csv', index_col='match_id')
# X_train = data.loc[:, 'start_time':'dire_first_ward_time'] # Drop information about future after 5 minutes passed

# columns_with_NANs = pd.isnull(X_train).any()[pd.isnull(X_train).any() == True].keys() # All columns with missing values
# print(" ".join(columns_with_NANs))

"""
1.1 Следующие признаки имеют пропуски в своих значениях: first_blood_time, first_blood_team, first_blood_player1,
first_blood_player2, radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward_time,
dire_bottle_time, dire_courier_time, dire_flying_courier_time, dire_first_ward_time. Очень просто объяснить пропущенные
значения в признаках first_blood_time и first_blood_team: эти пропуски обусловелны тем, что в течение первых 5 минут 
игры не произошло события First Blood, т. е. никто никого не убил.
"""

X_train_zeros = X_train.fillna(0)                # Filling missing values with zeros
X_train_infs = X_train.fillna(10**20)            # Filling missing values with big values  
X_train_means = X_train.fillna(X_train.mean())   # Filling missing values with mean values

y_train = data['radiant_win']

"""
1.2 Столбец, содержащий целевую переменную называется radiant_win.   
"""

kf = KFold(X_train.shape[0], n_folds=5, shuffle=True, random_state=1)

result = []
for X in [X_train_zeros, X_train_infs, X_train_means]:
    scores = {}
    times = {}
    for n in [10, 20, 30, 250]:
        start_time = time.time()
        clf = GradientBoostingClassifier(n_estimators=n, random_state=241)
        scores[n] = cross_val_score(clf, X, y_train, cv=kf, scoring="roc_auc", n_jobs=-1).mean()
        end_time = time.time()
        times[n] = end_time - start_time
    result.append([scores, times])
print(result)

"""
1.3 Кросс-валидация для градиентного бустинга с 30 деревьями проводилась 71 секунду, при этом, получившееся
качество ROC-AUC score составило 0.68990.
"""

"""
1.4 На мой взгляд есть смылсл пробовать брать больше 30 деревьев в градиентном бустинге. Например, когда я попробовал 
взять 250 деревьев, я получил оценку качества равную 0.71588. Для того, чтобы уменьшить время обичения при большом 
количестве деревьев можно уменьшать максимальную глубину деревьев или вместо всей обучающей выборки использовать
только её часть.
"""

scaler = StandardScaler()
X_train_zeros_scaled = scaler.fit_transform(X_train_zeros)     
X_train_infs_scaled = scaler.fit_transform(X_train_infs)
X_train_means_scaled = scaler.fit_transform(X_train_means)

def LogReg(X1, X2, X3):
    result = []
    for X in [X1, X2, X3]:
        scores = {}
        times = {}
        for C in [0.00001, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300]:
            start_time = time.time()
            clf = LogisticRegression(C=C)
            scores[C] = cross_val_score(clf, X, y_train, cv=kf, scoring="roc_auc", n_jobs=-1).mean()
            end_time = time.time()
            times[C] = end_time - start_time
        result.append([scores, times])
    return result

result = LogReg(X_train_zeros_scaled, X_train_infs_scaled, X_train_means_scaled)
print(result)

"""
2.1 Для данной модели наилучшее качество получилось при замене всех пропусков в признаках на их средние значения. 
При этом качество составило 0.71677. На качество предсказаний очень слабо вляет параметр "C" регуляризации, отличия
в качестве наблюдаются только в 3 знаке после запятой. Тем не менее наибольшего качества модель достигает при 
значении параметра C=0.003. Очевидна разница показателей качества при решении задачи с использованием градиентного
бустинга и логистической регрессии. Для 30 деревьев в градинтном бустинге эта разница составляет почти 3 процента.
Я думаю это связано с тем, что 30 деревьев мало и модель градиентного бустинга оказывается недообученной. Помимо
прочего логистическая регрессия работает заметно быстрее в сравнении с градиентным бустингом. Разница во времени
для модели логистической регрессии с наилучшим параметром C и модели градиентного бустинга с 30 деревьями составила
52 секунды.  
"""

X_all = []
for X in [X_train_zeros, X_train_infs, X_train_means]:   
    X = X.drop(X_train_zeros[['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                                            'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']], 1)
    X_scaled = scaler.fit_transform(X)
    X_all.append(X_scaled)
X_train_zeros_scaled, X_train_infs_scaled, X_train_means_scaled = X_all

result = LogReg(X_train_zeros_scaled, X_train_infs_scaled, X_train_means_scaled)
print(result)

"""
2.2 При изъятии категориальных признаков качество модели практически не изменилось. Максимальное значение качества
по-прежнему достигается при значении параметра C=0.003 и составляетс 0.71681. На мой взгляд этот факт объясняется
тем, что в "сыром" виде данные признаки не несут како-либо существенной информации, а лишь вносят дополнительные шумы. 
Необходимо смотреть на статистику всех героев в целом: какой герой, как часто выигрывает и т.д.
"""

heroes = X_train_zeros[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                          'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
idents = set()
for column in heroes.columns:
    idents |= set(heroes[column].values)
print(len(idents))
    
"""
2.3 В данной игре существует 108 различных идентификаторов героев.
"""

X_pick = sp.zeros((X_train.shape[0], len(idents)))
dict_idents = {ident: ind for ind, ident in zip(range(len(idents)), idents)}
for i, match_id in enumerate(X_train.index):
    for p in range(5):
        X_pick[i, dict_idents[data.ix[match_id, 'r%d_hero' % (p+1)]]] = 1
        X_pick[i, dict_idents[data.ix[match_id, 'd%d_hero' % (p+1)]]] = -1
        
X_train_zeros_scaled = sp.hstack((sp.array(X_train_zeros_scaled), X_pick))
X_train_infs_scaled = sp.hstack((sp.array(X_train_infs_scaled), X_pick))
X_train_means_scaled = sp.hstack((sp.array(X_train_means_scaled), X_pick))

result = LogReg(X_train_zeros_scaled, X_train_infs_scaled, X_train_means_scaled)
print(result)

"""
2.4 При dummy кодировании качество модели заметно повысилось и составило 0.75195, при этом наилучший параметр 
регуляризации составил C=0.1. Это объясняется тем, что теперь алгоритм получил важную информацию о героях,
смог учесть частоту побед тех или иных героев.
"""

X_test = pd.read_csv('features_test.csv', index_col='match_id')
X_test_means = X_test.fillna(X_test.mean())
X_test_means = X_test_means.drop(X_test_means[['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                                                             'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']], 1)
X_test_means_scaled = scaler.fit_transform(X_test_means)
X_pick_test = sp.zeros((X_test.shape[0], len(idents)))
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick_test[i, dict_idents[X_test.ix[match_id, 'r%d_hero' % (p+1)]]] = 1
        X_pick_test[i, dict_idents[X_test.ix[match_id, 'd%d_hero' % (p+1)]]] = -1
X_test_means_scaled = sp.hstack((sp.array(X_test_means_scaled), X_pick_test))
clf_final = LogisticRegression(C=0.1)
clf_final.fit(X_train_means_scaled, y_train)
pred = clf_final.predict_proba(X_test_means_scaled)[:, 1]
min_pred, max_pred = min(pred), max(pred)
print(min_pred, max_pred)

"""
2.5 Максимальное значение прогноза на тестовой выборке равно 0.99644. Минимальное значение прогноза на тестовой
выборке равно 0.00870.
"""